# Logistic Regression using La Classy

In [27]:
import { parse } from "https://deno.land/std@0.204.0/csv/parse.ts";
import {
  ClassificationReport,
  Matrix,
  useSplit,
  CategoricalEncoder,
} from "https://deno.land/x/vectorizer@v0.3.4/mod.ts";
import {
  SagSolver,
  softmaxActivation,
  rmsPropOptimizer,
  crossEntropy,
} from "https://deno.land/x/classylala@v1.0.0/mod.ts";


We first load our dataset `iris.csv`.

In [2]:
const data = parse(Deno.readTextFileSync("../datasets/iris.csv"));

Skip the first row (header).

In [3]:
data.shift()

[
  "sepal length",
  "sepal width",
  "petal length",
  "petal width",
  "class"
]

We can now get the predictor and target variables from the dataset.

In [16]:
const x = data.map((fl, i) => fl.slice(0, 4).map(Number));

const X = new Matrix<"f64">(Float64Array.from(x.flat()), [data.length])
X.slice(0, 10)

idx,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [14]:
const y_pre = data.map((fl) => fl[4]);
y_pre.slice(0, 10)

[
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa",
  "Iris-setosa"
]

Our target variables are all strings. In order to use them for classification, we convert them into categorical variables.

In [11]:
const encoder = new CategoricalEncoder()
const y = encoder.fit(y_pre).transform<"f64">(y_pre, "f64")
y.slice(0, 10)

idx,0,1,2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [12]:
[X.shape, y.shape]

[ [ 150, 4 ], [ 150, 3 ] ]

We now split our dataset for training and testing purposes. 

In [17]:
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  X,
  y
);
x_train.slice(0, 10)

idx,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.7,3.2,1.3,0.2
2,4.6,3.1,1.5,0.2
3,5,3.6,1.4,0.2
4,5.4,3.9,1.7,0.4
5,4.6,3.4,1.4,0.3
6,4.4,2.9,1.4,0.2
7,5.4,3.7,1.5,0.2
8,4.8,3.4,1.6,0.2
9,4.8,3,1.4,0.1


Now that we have prepared our inputs, we can initialize our solver. Since we are performing logistic regression, we use a SAG solver.

We use the `crossEntropy` loss function which is used for multinomial classification, `rmsprop` as our optimizer, and finally a `softmax` function to compute joint probabilities.

In [28]:
const solver = new SagSolver({
  loss: crossEntropy(),
  activation: softmaxActivation(),
  optimizer: rmsPropOptimizer(4, 3),
});


We can then train our model using the data we acquired.

Setting the learning rate to a small value is desirable. Since our dataset is pretty simple, we are training our model for 300 epochs with 20 minibatches.

In [31]:
solver.train(x_train, y_train, {
  learning_rate: 0.01,
  epochs: 300,
  n_batches: 20,
  patience: 10
});

The model is trained, now it is time to evaluate its performance on our testing dataset

In [32]:
const res = solver.predict(x_test)
res.shape

[ 45, 3 ]

In [33]:
res.row(0)

Float64Array(3) [
  0.9999999998572584,
  1.4274144924604957e-10,
  1.948289218474953e-20
]

The softmax function provides probabilities for the data point to belong to each of the classes. In our case, the three numbers in the array represent the probabilities of the first data point belonging to the classes `setosa`, `versicolor`, and `virginica` respectively.

We convert these into one-hot representations by taking the `argmax`.

In [34]:
let i = 0;
for (const row of res.rows()) {
  const max = row.reduce((acc, curr, i, arr) => arr[acc] > curr ? acc : i, 0)
  const newR = new Array(row.length).fill(0)
  newR[max] = 1
  res.setRow(i, newR)
  i += 1;
}
res.slice(0, 10)

idx,0,1,2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


We can use our encoder to convert the categorical variables into class labels.

In [35]:
const y_pred = encoder.untransform(res)
const y_act = encoder.untransform(y_test)

In [37]:
[y_pred, y_act]

[
  [
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-versicolor", "Iris-versicolor",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-virginica",  "Iris-versicolor", "Iris-virginica",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-versicolor", "Iris-versicolor", "Iris-versicolor",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica",
    "Iris-virginica",  "Iris-virginica",  "Iris-virginica"
  ],
  [
    "Iris-setosa",     "Iris-setosa",     "Iris-setosa",
    "Iris-setosa",     "Iris-setosa"

Finally, we can generate a classification report based on our results.

In [36]:
new ClassificationReport(y_act, y_pred)

Classification Report
Number of classes:	3
Class	Preci	F1	Rec	Sup
Iris-setosa	1	1	1	13
Iris-versicolor	1	1	1	17
Iris-virginica	0.8823529411764706	0.9375	1	15
Accuracy		0.9851851851851852	45

As we see, the classifier easily classifies different iris species. This is possible because the classes are easily separable. In a more complex database, these results may greatly vary.